## Import taxi_clean

In [43]:
import warnings
warnings.simplefilter("ignore")

import pandas as pd
import matplotlib as plt
import numpy as np
import time
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

#import data 
data = pd.read_csv("taxi_clean_med.csv")

#display data
print(data.shape)
      
data.head()

(8139, 19)


,trip_distance,fare_amount,winter,spring,summer,fall,PULongitude,PULatitude,DOLongitude,DOLatitude,pickup_datetime,dropoff_datetime,ride_duration,Early morning,Morning,Afternoon,Night,Holiday Proximity,label
0,1.00,9.0,0,0,0,1,-73.983310,40.737348,-73.990478,40.740313,2019-11-06 18:12:46,2019-11-06 18:25:12,0 days 00:12:26.000000000,0,0,0,0,0,C
1,1.13,6.0,0,1,0,0,-73.965691,40.768542,-73.965572,40.782460,2019-04-21 11:02:37,2019-04-21 11:06:58,0 days 00:04:21.000000000,0,1,0,0,0,B
2,2.24,10.5,0,0,1,0,-73.977432,40.758226,-73.983310,40.737348,2019-08-29 21:04:05,2019-08-29 21:17:43,0 days 00:13:38.000000000,0,0,0,0,0,D
3,0.70,6.0,0,0,1,0,-73.985929,40.748808,-73.978367,40.764425,2019-07-16 12:24:26,2019-07-16 12:31:31,0 days 00:07:05.000000000,0,0,0,0,0,B
4,2.18,8.5,1,0,0,0,-73.954568,40.765507,-73.971256,40.749172,2019-12-05 11:26:11,2019-12-05 11:33:22,0 days 00:07:11.000000000,0,1,0,0,0,C


In [44]:
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
data['pickup_datetime'] = pd.to_numeric(data['pickup_datetime'])
data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'])
data['dropoff_datetime'] = pd.to_numeric(data['dropoff_datetime'])

data['ride_duration'] = data['dropoff_datetime'] - data['pickup_datetime']

data.fillna(0, inplace= True)
print (data.dtypes)
data.head()

trip_distance        float64
fare_amount          float64
winter                 int64
spring                 int64
summer                 int64
fall                   int64
PULongitude          float64
PULatitude           float64
DOLongitude          float64
DOLatitude           float64
pickup_datetime        int64
dropoff_datetime       int64
ride_duration          int64
Early morning          int64
Morning                int64
Afternoon              int64
Night                  int64
Holiday Proximity      int64
label                 object
dtype: object


,trip_distance,fare_amount,winter,spring,summer,fall,PULongitude,PULatitude,DOLongitude,DOLatitude,pickup_datetime,dropoff_datetime,ride_duration,Early morning,Morning,Afternoon,Night,Holiday Proximity,label
0,1.00,9.0,0,0,0,1,-73.983310,40.737348,-73.990478,40.740313,1573063966000000000,1573064712000000000,746000000000,0,0,0,0,0,C
1,1.13,6.0,0,1,0,0,-73.965691,40.768542,-73.965572,40.782460,1555844557000000000,1555844818000000000,261000000000,0,1,0,0,0,B
2,2.24,10.5,0,0,1,0,-73.977432,40.758226,-73.983310,40.737348,1567112645000000000,1567113463000000000,818000000000,0,0,0,0,0,D
3,0.70,6.0,0,0,1,0,-73.985929,40.748808,-73.978367,40.764425,1563279866000000000,1563280291000000000,425000000000,0,0,0,0,0,B
4,2.18,8.5,1,0,0,0,-73.954568,40.765507,-73.971256,40.749172,1575545171000000000,1575545602000000000,431000000000,0,1,0,0,0,C


## Base Model (rate x distance) and its RMSE

In [45]:
train, test = train_test_split(data, test_size=0.33, random_state=42)
import numpy as np

rate = data['fare_amount'].mean() / data['trip_distance'].mean()

def compute_rmse(actual, predicted):
  return np.sqrt(np.mean((actual - predicted)**2))

def print_rmse(taxi, rate, name):
  print("RMSE = {}".format(compute_rmse(taxi['fare_amount'], rate * taxi['trip_distance'])), name)
  
print("Rate = ${}/miles".format(rate))
print_rmse(train, rate, 'Train')
print_rmse(test, rate, 'Test')

Rate = $4.286638394340321/miles
RMSE = 6.688017111190376 Train
RMSE = 6.481507639558073 Test


## Random Forest

In [46]:
y = data['fare_amount']
X = data.drop(columns=['fare_amount','label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

random_forest = RandomForestRegressor(n_estimators=200, max_depth=5, max_features= 4, random_state=42)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)

print("Test RMSE: ", mean_squared_error(y_test, y_pred) ** 0.5)

Test RMSE:  2.3824010418746435


## Decision Trees

In [56]:
decision_tree = DecisionTreeRegressor(max_depth=5, max_features= 4, random_state=42)
decision_tree.fit(X_train, y_train)
y_pred1 = decision_tree.predict(X_test)

print("Test RMSE: ", mean_squared_error(y_test, y_pred1) ** 0.5)

Test RMSE:  6.021622956210214
